In [1]:
#====================================
# Author: gmd8si
# Date: 16, November, 2016
#====================================
%matplotlib inline
import tensorflow as tf
import numpy as np
import pandas as pd
from detector_dr import Detector
from util import load_image, crop_image, augment           # Questionalable since we can use our own loading pipeline
import os
import matplotlib.pyplot as plt
import csv
import time
#================================================
# Setting Hyper-Parameters 
#================================================

#weight_path = '../data/caffe_layers_value.pickle'
#model_path = '../models/caltech256/'
PRETRAINED_MODEL_PATH= '/home/gmd8si/Weakly_detector/src/ckpt_dir/model.ckpt-0'  #None #'../models/caltech256/model-0'
N_EPOCHS = 10000
INIT_LEARNING_RATE = 0.01
WEIGHT_DECAY_RATE = 0.0005
MOMENTUM = 0.9
IMAGE_HEIGHT  = 512    #960
IMAGE_WIDTH   = 512    #720
NUM_CHANNELS  = 3
BATCH_SIZE = 10
N_LABELS = 5
DROPOUT = 0.50  
LOGS_PATH = './tensorflow_logs/new'
WEIGHT_PATH = None
TRAINSET_PATH = '/home/gmd8si/Weakly_detector/data/customKaggle512_train.csv'
VALSET_PATH ='/home/gmd8si/Weakly_detector/data/customKaggle512_val.csv'

# Reading Data

In [2]:
#=======================================================================================================
# Reading Training data from CSV FILE
#=======================================================================================================

csv_path = tf.train.string_input_producer([TRAINSET_PATH], shuffle=True)
textReader = tf.TextLineReader()
_, csv_content = textReader.read(csv_path)
im_name, im_label = tf.decode_csv(csv_content, record_defaults=[[""], [1]])

im_content = tf.read_file(im_name)
train_image = tf.image.decode_jpeg(im_content, channels=3)
#train_image = crop_image(train_image)   # Will do it again
train_image = tf.cast(train_image, tf.float32) / 255. # could be unnecessary
size = tf.cast([512, 512], tf.int32)
train_image = tf.image.resize_images(train_image, size)
train_image = augment(train_image)
#label = tf.one_hot(label, 5, 1, 0 )
train_label = tf.cast(im_label, tf.int64) # unnecessary
train_image_batch, train_label_batch = tf.train.batch([train_image, train_label], batch_size=BATCH_SIZE)

In [3]:
#=======================================================================================================
# Reading Validation data from CSV FILE
#=======================================================================================================

val_csv_path = tf.train.string_input_producer([VALSET_PATH], shuffle=True) # No need to set no. epochs here
val_textReader = tf.TextLineReader()
_, val_content = val_textReader.read(val_csv_path)
val_image, val_label = tf.decode_csv(val_content, record_defaults=[[""], [1]])

val_image_content = tf.read_file(val_image)
val_image = tf.image.decode_jpeg(val_image_content, channels=3)
val_image = tf.cast(val_image, tf.float32) / 255. # could be unnecessary
size = tf.cast([512, 512], tf.int32)
val_image = tf.image.resize_images(val_image, size)
val_image = tf.image.per_image_whitening(val_image)
val_label = tf.cast(val_label, tf.int64) # unnecessary
val_image_batch, val_label_batch = tf.train.batch([val_image, val_label], batch_size=BATCH_SIZE)

# Determining datasets length

In [4]:
trainfile = open(TRAINSET_PATH, "rb")
reader = csv.reader(trainfile)
trainset = sum(1 for row in reader)
print trainset
trainfile.close()

valfile = open(VALSET_PATH, "rb")
reader = csv.reader(valfile)
valset = sum(1 for row in reader)
print valset
valfile.close()

21587
1671


# Placeholder

In [5]:
learning_rate = tf.placeholder( tf.float32, [])
images_tf = tf.placeholder( tf.float32, [None, 512, 512, 3], name="images")
labels_tf = tf.placeholder( tf.int64, [None], name='labels')  # the dimensions could be [None,N_CLASSES]

# Optimizer Properties

In [6]:
detector = Detector(N_LABELS)

In [7]:
p1,p2,p3,p4,conv5, conv6, gap, output = detector.inference(images_tf, DROPOUT)
loss_tf = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits( output, labels_tf ))

In [8]:
weights_only = filter( lambda x: x.name.endswith('W:0'), tf.trainable_variables() )
weight_decay = tf.reduce_sum(tf.pack([tf.nn.l2_loss(x) for x in weights_only])) * WEIGHT_DECAY_RATE
loss_tf += weight_decay

In [9]:
optimizer = tf.train.MomentumOptimizer( learning_rate, MOMENTUM)
grads_and_vars = optimizer.compute_gradients( loss_tf )
grads_and_vars = map(lambda gv: (gv[0], gv[1]) if ('conv6' in gv[1].name or 'GAP' in gv[1].name) else (gv[0]*0.1, gv[1]), 
                     grads_and_vars)
#grads_and_vars = [(tf.clip_by_value(gv[0], -5., 5.), gv[1]) for gv in grads_and_vars]
train_op = optimizer.apply_gradients( grads_and_vars )

# Accuracy

In [10]:
#label_predictions = output.argmax(axis=1)
#accuracy = (label_predictions == labels_tf).sum()
# check if we have access to labels_tf in session, like the train.caltech

correct_pred = tf.equal(tf.argmax(output, 1), labels_tf)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Tensorflow checkpoint saver

In [11]:
# Saver Operation to save and restore all variables, first create directory
ckpt_dir = "./ckpt_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)
saver = tf.train.Saver(max_to_keep=50)   #Indicates max number of recent checkpoint files to keep

In [12]:
if not os.path.exists('./results'):
    os.makedirs('./results')
f_log = open('./results/customVGGCAM.txt', 'w')


# Training and Validation Sessions

In [ ]:
with tf.Session() as sess:
    
    if PRETRAINED_MODEL_PATH:
        print "Using Pretrained model"
        saver.restore(sess, PRETRAINED_MODEL_PATH)
    else:    
        sess.run(tf.initialize_all_variables())    
    #sess.run(tf.initialize_local_variables())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    loss_list = []
    process_start_time= time.time()
    
    for epoch in range(300):    
        
        epoch_start_time = time.time()
        for steps in range( trainset/BATCH_SIZE +1):
            train_imbatch, train_labatch = sess.run([train_image_batch, train_label_batch])
            _, loss_val, output_val, acc = sess.run([train_op, loss_tf, output, accuracy], feed_dict={
                    learning_rate: INIT_LEARNING_RATE, images_tf: train_imbatch, labels_tf: train_labatch})
                
            loss_list.append(loss_val)                              # This must be for visualizing loss curve 
            
            if (steps) % 5 == 0:   # after 5 batches
                print "======================================"
                print "Epoch", epoch+1, "Iteration", steps
                print "Processed", (steps*BATCH_SIZE), '/', trainset
                label_predictions = output_val.argmax(axis=1)
                acc = (label_predictions == train_labatch).sum()
                print "Accuracy per batch:", acc
                #print "Accuracy Percentage:", (acc/len(current_labels))*100
                print "Training Loss:", np.mean(loss_list)
                loss_list = []                
            
        n_correct = 0
        n_data = 0
        
        for i in range (valset/BATCH_SIZE +1):
            
            val_imbatch, val_labatch = sess.run([val_image_batch, val_label_batch])
            output_vals = sess.run(output, feed_dict={images_tf:val_imbatch, labels_tf: val_labatch})

            label_predictions = output_vals.argmax(axis=1)
            acc = (label_predictions == val_labatch).sum()

            n_correct += acc                  # No of correctly idenified examples in whole val set
            n_data += BATCH_SIZE

        acc_all = n_correct / float(n_data)
        f_log.write('epoch:'+str(epoch+1)+'\tacc:'+str(acc_all) + '\n')
        print "===========**VALIDATION ACCURACY**================"
        print 'epoch:'+str(epoch+1)+'\tacc:'+str(acc_all) + '\n'
        print 'Time Elapsed for Epoch:'+str(epoch+1)+' is '+str(time.time() - epoch_start_time)
        INIT_LEARNING_RATE *= 0.99
        
        if (epoch % 20 == 0):
            saver.save(sess, ckpt_dir + "/model-customVGGCAM.ckpt", global_step=epoch)
            print 'HERE'
        

Using Pretrained model
